**Donwload torchtext this versiont**

In [1]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/
!pip install torchtext==0.10.1

Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/README  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.sense  
  inflating: /usr/share/nltk_data

In [99]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import re
import string
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import emoji
import torchtext
import random
import pandas as pd
from torch.utils.data import DataLoader
import torch
import torch.nn.functional as F
import torchtext
import time
import random
import pandas as pd
from torchtext.legacy import data
from sklearn.metrics import classification_report

torch.manual_seed(1)

**GPU support available or not**

In [100]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


**HyperParameter**

In [101]:
LEARNING_RATE = 0.0003
BATCH_SIZE = 128
NUM_EPOCHS = 15
EMBEDDING_DIM = 128
HIDDEN_DIM = 256
NUM_CLASSES = 2

**Preparing Data**

In [102]:
df = pd.read_csv('/kaggle/input/extradataset/edos_labelled_aggregated.csv')

# for task A we don't need these things
df.drop(['rewire_id','label_category','label_vector'], axis=1, inplace=True)

dfTrain=df[df['split']=="train"]
dfVal=df[df['split']=="dev"]
dfTest=df[df['split']=="test"]
#df.drop(['rewire_id','label_category','label_vector'], axis=1, inplace=True)

dfTrain.drop(['split'], axis=1, inplace=True)
dfVal.drop(['split'], axis=1, inplace=True)
dfTest.drop(['split'], axis=1, inplace=True)

print(dfTrain.head())

dfTrain.to_csv('train.csv',index=False)
dfTest.to_csv('test.csv',index=False)
dfVal.to_csv('val.csv',index=False)

                                                text label_sexist
1                            Then, she's a keeper. 😉   not sexist
2  This is like the Metallica video where the poo...   not sexist
3                                             woman?   not sexist
5  Unlicensed day care worker reportedly tells co...   not sexist
6  [USER] Leg day is easy. Hot girls who wear min...       sexist


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [103]:
TEXT = data.Field(
    tokenize='spacy', # default splits on whitespace
    tokenizer_language='en_core_web_sm'
)

### Defining the label processing
LABEL = data.LabelField(dtype=torch.long)

**Loading Data**

In [104]:
fields = [('text', TEXT), ('label_sexist', LABEL)]
train_data = data.TabularDataset(path='/kaggle/working/train.csv', format='csv',skip_header=True, fields=fields)

In [105]:
test_data = data.TabularDataset(path='/kaggle/working/test.csv', format='csv',skip_header=True, fields=fields)
valid_data = data.TabularDataset(path='/kaggle/working/val.csv', format='csv',skip_header=True, fields=fields)

In [106]:
print(vars(train_data.examples[0]))

{'text': ['Then', ',', 'she', "'s", 'a', 'keeper', '.', '😉'], 'label_sexist': 'not sexist'}


**Data Cleaning**: Not used

In [107]:
class DataCleaning():
    """
        Take a list of strings and preprocess strings, it preforms:
        
    """
    def __init__(self, list_data):
        self.data = list_data
        self.len = len(list_data)

    def clean_data(self):
        self.emojis_to_text()
        self.lowerCase()
        self.linkToTag()
        self.removePunctuations()
        self.removeWordsWithNumber()
        self.rootWord(lemmatizer=True)
        self.removeStopword()
        return self.data
        
    def tokenizer():
        pass
        
    def rootWord(self,lemmatizer=False):
        """
            Steamer is faster than lammatization.
            Steamer cut the last few words, and use the root word
            lemmatizer convert many form a word to the same word. 
            stemmer will retun 'car' for word 'caring'
            while lemmatizer return 'care' for 'caring'
        """
        if not lemmatizer:
            stemmer = PorterStemmer()
            for i in range(self.len):
                self.data[i]=" ".join([stemmer.stem(word) for word in self.data[i].split()])
        else:
            lemmatizer = WordNetLemmatizer()
            for i in range(self.len):
                self.data[i]=" ".join([lemmatizer.lemmatize(word) for word in self.data[i].split()])
        
    def removePunctuations(self):
        removePunc = re.compile(r'[^\w\s]')
        for i in range(self.len):
            self.data[i] = re.sub(removePunc, r" ",self.data[i])

    def emojis_to_text(self):
        """
            Converting image to its text meaning.
            Format: 👍 to ":thumbs_up:"
        """
        for i in range(self.len):
            self.data[i]=emoji.demojize(self.data[i])
            
    def linkToTag(self):
        """replacing web links with '<URL>'"""
        linkRegex = re.compile(r'\b(www|http|https)[^ |\n]*')

        for i in range(self.len):
            self.data[i] = re.sub(linkRegex, r"<URL>",self.data[i])
    
    def removeWordsWithNumber(self):
        """
            Zero or more number of non-whitespace then digit, 
            then Zero or more number of non-whitespace
        """
        wordNumRegex = re.compile(r'\S*\d\S*')
        for i in range(self.len):
            self.data[i]=re.sub(wordNumRegex,r"",self.data[i])
    
    def lowerCase(self):
        """
            It is sometimes important to keep the letter capital 
            as it signifies shouting in form of that word, but it
            depends on use case
        """
        for i in range(self.len):
            self.data[i]=self.data[i].lower()
            
    def removeStopword(self):
        """ Remove stopward from a string"""
        stop = stopwords.words('english')
        for i in range(self.len):
            self.data[i]=' '.join([word for word in self.data[i].split() if word not in stop])

In [108]:
# a=DataCleaning(dfTrain['text'].tolist())
# trainTexts=a.clean_data()
# a=DataCleaning(dfTest['text'].tolist())
# testTexts=a.clean_data()
# a=DataCleaning(dfVal['text'].tolist())
# valTexts=a.clean_data()

In [109]:
# trainText=dfTrain['text'].tolist()
# testText=dfTest['text'].tolist()
# valText=dfVal['text'].tolist()

In [110]:
len(train_data)

14000

**Build vocab**

In [111]:
TEXT.build_vocab(train_data)
LABEL.build_vocab(train_data)

print(f'Vocabulary size: {len(TEXT.vocab)}')
print(f'Number of classes: {len(LABEL.vocab)}')

Vocabulary size: 26887
Number of classes: 2


In [112]:
print(LABEL.vocab.stoi)

defaultdict(None, {'not sexist': 0, 'sexist': 1})


**Dataloader**

In [113]:
train_loader, valid_loader, test_loader = \
    data.BucketIterator.splits(
        (train_data, valid_data, test_data),
         batch_size=BATCH_SIZE,
         sort_within_batch=False,
         sort_key=lambda x: len(x.text),
         device=device
    )

**Shape of data**

In [114]:
print('Train')
for batch in train_loader:
    print(f'Text matrix size: {batch.text.size()}')
    print(f'Target vector size: {batch.label_sexist.size()}')
    break
    
print('\nValid:')
for batch in valid_loader:
    print(f'Text matrix size: {batch.text.size()}')
    print(f'Target vector size: {batch.label_sexist.size()}')
    break
    
print('\nTest:')
for batch in test_loader:
    print(f'Text matrix size: {batch.text.size()}')
    print(f'Target vector size: {batch.label_sexist.size()}')
    break

Train
Text matrix size: torch.Size([53, 128])
Target vector size: torch.Size([128])

Valid:
Text matrix size: torch.Size([9, 128])
Target vector size: torch.Size([128])

Test:
Text matrix size: torch.Size([6, 128])
Target vector size: torch.Size([128])


**Module**

In [115]:
class RNN(torch.nn.Module):
    
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()

        self.embedding = torch.nn.Embedding(input_dim, embedding_dim)
        #self.rnn = torch.nn.RNN(embedding_dim,
        #                        hidden_dim,
        #                        nonlinearity='relu')
        self.rnn = torch.nn.LSTM(embedding_dim,
                                 hidden_dim)        
        
        self.fc1 = torch.nn.Linear(hidden_dim, hidden_dim//2)
        self.fc2 = torch.nn.Linear(hidden_dim//2, output_dim)
        

    def forward(self, text):
        # text dim: [sentence length, batch size]
        
        embedded = self.embedding(text)
        # embedded dim: [sentence length, batch size, embedding dim]
        
        output, (hidden, cell) = self.rnn(embedded)
        # output dim: [sentence length, batch size, hidden dim]
        # hidden dim: [1, batch size, hidden dim]

        hidden.squeeze_(0)
        # hidden dim: [batch size, hidden dim]
        
        output = self.fc1(hidden)
        output = self.fc2(F.relu(output))
        return output

In [116]:
model = RNN(input_dim=len(TEXT.vocab),
            embedding_dim=EMBEDDING_DIM,
            hidden_dim=HIDDEN_DIM,
            output_dim=2 # could use 1 for binary classification
)

model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

**Accuracy Report**

In [117]:
def compute_accuracy(model, data_loader, device,report=False):

    predict=[]
    actual=[]
    
    with torch.no_grad():

        correct_pred, num_examples = 0, 0

        for i, (features, targets) in enumerate(data_loader):

            features = features.to(device)
            targets = targets.float().to(device)

            logits = model(features)
            _, predicted_labels = torch.max(logits, 1)

            num_examples += targets.size(0)
            correct_pred += (predicted_labels == targets).sum()
            
            predict+=predicted_labels.tolist()                         
            actual+=targets.tolist()
            
        if report:    
            print(classification_report(actual, predict, labels=[0,1]))
            
    return correct_pred.float()/num_examples * 100

In [118]:
DEVICE=device

**Training**

In [119]:
start_time = time.time()

for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx, batch_data in enumerate(train_loader):
        
        text = batch_data.text.to(device)
        labels = batch_data.label_sexist.to(device)

        ### FORWARD AND BACK PROP
        logits = model(text)
        loss = F.cross_entropy(logits, labels)
        optimizer.zero_grad()
        
        loss.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
                   f'Batch {batch_idx:03d}/{len(train_loader):03d} | '
                   f'Loss: {loss:.4f}')

    torch.save(model.state_dict(), 'lstm'+str(epoch)+".pt")
            
    with torch.set_grad_enabled(False):
        print(f'training accuracy: '
              f'{compute_accuracy(model, train_loader, device):.2f}%'
              f'\nvalid accuracy: '
              f'{compute_accuracy(model, valid_loader, device):.2f}%')
        
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
print(f'Test accuracy: {compute_accuracy(model, test_loader, device):.2f}%')

Epoch: 001/015 | Batch 000/110 | Loss: 0.6449
Epoch: 001/015 | Batch 050/110 | Loss: 0.5381
Epoch: 001/015 | Batch 100/110 | Loss: 0.5674
training accuracy: 75.73%
valid accuracy: 75.70%
Time elapsed: 0.03 min
Epoch: 002/015 | Batch 000/110 | Loss: 0.5638
Epoch: 002/015 | Batch 050/110 | Loss: 0.5237
Epoch: 002/015 | Batch 100/110 | Loss: 0.5357
training accuracy: 75.73%
valid accuracy: 75.70%
Time elapsed: 0.06 min
Epoch: 003/015 | Batch 000/110 | Loss: 0.5640
Epoch: 003/015 | Batch 050/110 | Loss: 0.5170
Epoch: 003/015 | Batch 100/110 | Loss: 0.4698
training accuracy: 75.73%
valid accuracy: 75.70%
Time elapsed: 0.09 min
Epoch: 004/015 | Batch 000/110 | Loss: 0.5251
Epoch: 004/015 | Batch 050/110 | Loss: 0.6481
Epoch: 004/015 | Batch 100/110 | Loss: 0.5014
training accuracy: 75.73%
valid accuracy: 75.70%
Time elapsed: 0.12 min
Epoch: 005/015 | Batch 000/110 | Loss: 0.4784
Epoch: 005/015 | Batch 050/110 | Loss: 0.5923
Epoch: 005/015 | Batch 100/110 | Loss: 0.5910
training accuracy: 75.

**Loading the best Model**

In [120]:
path = "/kaggle/working/lstm12.pt"
model.load_state_dict(torch.load(path))

<All keys matched successfully>

**F1 score at the test data**

In [121]:
with torch.set_grad_enabled(False):
    print('test accuracy: ',compute_accuracy(model, test_loader, device,True))

              precision    recall  f1-score   support

           0       0.85      0.85      0.85      3030
           1       0.54      0.54      0.54       970

    accuracy                           0.78      4000
   macro avg       0.70      0.70      0.70      4000
weighted avg       0.78      0.78      0.78      4000

test accuracy:  tensor(77.7750, device='cuda:0')


Macro F1 score is 0.70

In [122]:
with torch.set_grad_enabled(False):
    print('test accuracy: ',compute_accuracy(model, test_loader, device,True))

              precision    recall  f1-score   support

           0       0.85      0.85      0.85      3030
           1       0.54      0.54      0.54       970

    accuracy                           0.78      4000
   macro avg       0.70      0.70      0.70      4000
weighted avg       0.78      0.78      0.78      4000

test accuracy:  tensor(77.7750, device='cuda:0')


Reference

1. https://www.youtube.com/watch?v=CrS-LFXEiyk
2. https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html
3. https://github.com/rasbt/stat453-deep-learning-ss21/blob/main/L15/1_lstm.ipynb